<a href="https://colab.research.google.com/github/majdsuleiman98/Wikipedia-Chatbot-with-RAG-System/blob/main/ChatWithWikipedia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers langchain langchain_community wikipedia faiss-cpu

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.7/408.7 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 1.3 MB/s eta 0:00:00
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11679 sha256=eb90919ac7bb9344fb7d1fa71ceca69ee75ec06552f75fed9739aa60d85451ef
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36
  Attempting uninstall: lan

In [2]:
import nltk
nltk.download('punkt')
import faiss
import wikipedia
from copy import deepcopy
from transformers import pipeline
from langchain.chains import ConversationChain
from langchain.llms import HuggingFacePipeline
from langchain.memory import ConversationSummaryMemory
from sentence_transformers import SentenceTransformer
from langchain.document_loaders import WikipediaLoader
from langchain.text_splitter import NLTKTextSplitter
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
model_id = "google/flan-t5-large"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(
    model_id,
    device_map="auto",
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [4]:
pipe = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=32,
    num_beams=5,
    temperature=0.7,
    do_sample=True,
)
llm = HuggingFacePipeline(pipeline=pipe)

<ipython-input-4-2cd5f0ce6a3b>:10: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


In [5]:
def load_wikipedia_page(topic):
    try:
        data = WikipediaLoader(topic,load_max_docs=1,doc_content_chars_max=20_000).load()
        text = data[0].page_content
        metadata = {"title":data[0].metadata["title"],"source":data[0].metadata["source"]}
        return text, metadata

    except wikipedia.exceptions.DisambiguationError as e:
        return f"Your question could refer to multiple topics: {e.options[:3]}."

    except wikipedia.exceptions.PageError:
        return "Sorry, I couldn't find information on that topic."

In [6]:
def split_to_chunks(text,metadata):
    chunks = NLTKTextSplitter(
        chunk_size=400,
        chunk_overlap=0).create_documents([text],metadatas=[metadata])
    return chunks

In [7]:
def encode_texts(model_id,chunks):
    embedding_model = SentenceTransformer(model_id,device="cpu")
    doc_text = [chunk.page_content for chunk in chunks]
    embeddings = embedding_model.encode(doc_text,show_progress_bar=True)
    return embeddings

In [8]:
def create_ids(chunks):
    ids = [id for id in range(len(chunks))]
    return ids

In [9]:
def build_vectordb(embeddings,ids, dim):
    normalized_embeddings = deepcopy(embeddings)
    faiss.normalize_L2(normalized_embeddings)
    faiss_index = faiss.IndexIDMap(faiss.IndexFlatIP(dim))
    faiss_index.add_with_ids(normalized_embeddings,ids)
    return faiss_index

In [10]:
def builder(topic):
    text,metadata = load_wikipedia_page(topic)
    chunks = split_to_chunks(text,metadata)
    ids = create_ids(chunks)
    embeddings = encode_texts(model_id,chunks)
    dim = len(embeddings[0])
    faiss_index = build_vectordb(embeddings,ids, dim)
    return faiss_index,chunks

In [11]:
def extract_context(results,chunks):
    context = ""
    for id in results[1][0]:
        context += chunks[id].page_content
    return context

In [12]:
def similarity_search(query,faiss_index, chunks,top_k=5):
    embedding_model = SentenceTransformer(model_id,device="cpu")
    query_encode = embedding_model.encode([query])
    faiss.normalize_L2(query_encode)
    results = faiss_index.search(query_encode,top_k)
    context = extract_context(results,chunks)
    return context

In [13]:
topic = input("Please enter your topic : ")
faiss_index, chunks = builder(topic)

Please enter your topic : lionel messi


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

In [14]:
memory = ConversationSummaryMemory(llm=llm)
conversation = ConversationChain(llm=llm, memory=memory, verbose=True)
while True:
    query = input("Please enter your query: ")
    context = similarity_search(query, faiss_index, chunks)
    if query == "stop":
      break
    if query == "clean":
      conversation.memory.clear()
      continue
    prompt = (
        "Use the following context to answer user question.\n\n"
        f"Context:\n{context}\n\n"
        f"Question: {query}?\n\n"
        "Answer:"
    )
    response = conversation.run(prompt)
    print("Response:", response)

<ipython-input-14-efad2d72fd92>:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryMemory(llm=llm)
<ipython-input-14-efad2d72fd92>:2: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  conversation = ConversationChain(llm=llm, memory=memory, verbose=True)


Please enter your query: When was messi born?


<ipython-input-14-efad2d72fd92>:17: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = conversation.run(prompt)




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Use the following context to answer user question.

Context:
A prolific goalscorer and creative playmaker, Messi has scored over 850 senior career goals for club and country.

Born in Rosario, Argentina, Messi relocated to Spain to join Barcelona at age 13, and made his competitive debut at age 17 in October 2004.Lionel Andrés Messi (Spanish pronunciation: [ljoˈnel anˈdɾes ˈmesi] ; born 24 June 1987), also known as Leo Messi, is an Argentine professional footballer who plays as a forward for and captains both Major League Soccer club Inter Miami and the Argentina national team.He made his La Liga debut for Barcelona on 16 October 2004 against Espanyol, and scor

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


...

Token indices sequence length is longer than the specified maximum sequence length for this model (590 > 512). Running this sequence through the model will result in indexing errors



> Finished chain.
Response: eight
Please enter your query: When did the first one of them win?


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Lionel Messi set numerous records for individual accolades won throughout his professional footballing career such as eight Ballon d'Or awards and eight times
Human: Use the following context to answer user question.

Context:
He made his La Liga debut for Barcelona on 16 October 2004 against Espanyol, and scored his first senior goal on 1 May 2005 against Albacete, from an assist by Ronaldinho, becoming at that time the youngest-ever scorer for the club.He finished the campaign having scored for four of his five teams with a total of 36 goals in all official competitions.During the 2011–12 season, he set La Liga and European records for most goals in a season, while e

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
